In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
data = pd.read_csv("../input/gtd/globalterrorismdb_0718dist.csv",encoding="ISO-8859-1")

In [ ]:
data

In [ ]:
nd=data[["vicinity","crit1","crit2","crit3","multiple","suicide","property","ishostkid","ransom","targtype1","attacktype1","weaptype1"]]

In [ ]:
nd["ransom"].fillna(0)
nd["ishostkid"].fillna(0)

In [ ]:

for i in range(181691):
    if (nd["ransom"][i]!=0.0) and (nd["ransom"][i]!=1):
        nd["ransom"][i]=0
    if (nd["vicinity"][i]!=0.0) and (nd["vicinity"][i]!=1):
        nd["vicinity"][i]=0
    if (nd["ishostkid"][i]!=0.0) and (nd["ishostkid"][i]!=1):
        nd["ishostkid"][i]=0

nd.columns

In [ ]:
nd["ishostkid"].unique()

In [ ]:
cat_vars=['targtype1','attacktype1','weaptype1']
for var in cat_vars:
    cat_list='var'+'_'+var
    cat_list = pd.get_dummies(nd[var], prefix=var)
    data1=nd.join(cat_list)
    nd=data1

In [ ]:
nd.columns
suc=data[["success"]]
suc.shape

In [ ]:
nd.columns.values

In [ ]:
X = nd.loc[:, nd.columns != 'success']

In [ ]:
y = suc.loc[:, suc.columns == 'success']

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [ ]:
os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
columns = X_train.columns

In [ ]:
os_data_X,os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['success'])
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print("Number of no subscription in oversampled data",len(os_data_y[os_data_y['success']==0]))
print("Number of subscription",len(os_data_y[os_data_y['success']==1]))
print("Proportion of no subscription data in oversampled data is ",len(os_data_y[os_data_y['success']==0])/len(os_data_X))
print("Proportion of subscription data in oversampled data is ",len(os_data_y[os_data_y['success']==1])/len(os_data_X))

In [ ]:
os_data_y.shape

In [ ]:
data_final_vars=nd.columns.values.tolist()
y=['success']
X=[i for i in data_final_vars if i not in y]
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
os_data_X.columns

In [ ]:
arr=[]
for i in range(len(os_data_X.columns)):
    if(rfe.support_[i]==True):
        arr.append(os_data_X.columns[i])

In [ ]:
Xt=os_data_X[arr]
Xt.shape

In [ ]:
yt=os_data_y
yt.shape

In [ ]:
import statsmodels.api as sm
logit_model=sm.Logit(yt,Xt)
result=logit_model.fit()
print(result.summary2())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(Xt, yt, test_size=0.3, random_state=0)
logreg = LogisticRegression()
logreg.fit(Xt, yt)

In [ ]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

In [ ]:
yt.shape

In [ ]:
cnt=0
for i in range(len(yt["success"])):
    if(yt["success"][i]==1):
        cnt=cnt+1
print(cnt)

In [169]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[17674 16345]
 [ 5181 28705]]


In [170]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.52      0.62     34019
           1       0.64      0.85      0.73     33886

    accuracy                           0.68     67905
   macro avg       0.71      0.68      0.67     67905
weighted avg       0.71      0.68      0.67     67905



In [171]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

NameError: name 'plt' is not defined